In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from datetime import datetime
import pickle
import re
import scipy

%matplotlib inline

test_flag = 0
history_len = 7

In [2]:
with open('..//datasets/ts_df_train_list_updated.pickle','rb') as f:
    list_dfs_per_length = pickle.load(f)
df_ts_features = list_dfs_per_length[history_len-2]

In [3]:
def clean_name(name):
    seperate_loc = name.find('-')
    if seperate_loc == -1:
        pass
    else:
        name = name[:seperate_loc-1]
    return name

def base_feature_name(col_names):
    names_proccesed = []
    for name in col_names:
        names_proccesed.append(clean_name(name))
    return list(set(names_proccesed))

if test_flag:
    print(base_feature_name(df_ts_features.columns))
    
def indexes_per_name(col_names,name):
    return [x for x in col_names if x[:len(name)]==name]

if test_flag:
    print(indexes_per_name(df_ts_features.columns,'Persist. value_tmax_1'))
    
unique_names = base_feature_name(df_ts_features.columns)
col_idx_list = [indexes_per_name(df_ts_features.columns,name) for name in unique_names]
col_idx_list = [x for x in col_idx_list if len(x)>1]

In [64]:
# Function API for time series features
# row - pandas row from Dataframe with time searies
# col_idx_list - created in cell above
# series_functions - list of functions that recive an ndarray of shape (days_back,) and returns a number
def time_features_per_row(row,col_idx_list,series_functions):
    time_features = []
    feature_name = []
    for col_idxs in col_idx_list:
        time_series = row[col_idxs].values
        for func in series_functions:
            feature_name.append(clean_name(col_idxs[0]) + '_' + func.__name__)
            if len(time_series) == history_len:
                time_features.append(func(time_series))
            else:
                time_features.append(np.nan)
        
    return (feature_name,time_features)

if test_flag:
    print(time_features_per_row(df_ts_features.iloc[0],col_idx_list,[np.std]))

## sequential functions

In [65]:
def mean_no_outliars(series):
    series = np.sort(series)
    return np.mean(series[1:-1])

In [66]:
# slope of linfit to series
# (in future can principe retrurn error evaluations)
def trend(series):
    return scipy.stats.linregress(range(len(series)), series).slope

In [67]:
def rms_daily_change(series):
    return np.sqrt(np.mean(np.diff(series)**2))

In [68]:
series_functions = [np.max,np.min,np.std,np.mean,np.median,trend,mean_no_outliars,rms_daily_change]


In [152]:
def single_row_df(row):
    (feature_name,feature_value_list) = time_features_per_row(row,col_idx_list,series_functions)
    dat_val = row.iloc[0]
    return pd.DataFrame(data = np.array([dat_val] + feature_value_list).reshape(1,-1),columns=['Validiy data']+feature_name)

In [153]:
single_row_df(df_ts_features.iloc[3])

,Validiy data,Persist. value_wind_2_amax,Persist. value_wind_2_amin,Persist. value_wind_2_std,Persist. value_wind_2_mean,Persist. value_wind_2_median,Persist. value_wind_2_trend,Persist. value_wind_2_mean_no_outliars,Persist. value_wind_2_rms_daily_change,CO_tmax_2_amax,...,EC_tmax_2_mean_no_outliars,EC_tmax_2_rms_daily_change,Persist. value_tmax_1_amax,Persist. value_tmax_1_amin,Persist. value_tmax_1_std,Persist. value_tmax_1_mean,Persist. value_tmax_1_median,Persist. value_tmax_1_trend,Persist. value_tmax_1_mean_no_outliars,Persist. value_tmax_1_rms_daily_change
0,1.514412e+09,18.0,8.0,3.522522,11.857143,11.0,-0.607143,11.4,5.552777,26.0,...,19.4,3.41565,24.0,18.0,2.13809,21.0,20.0,0.821429,21.0,1.95789


In [159]:
list_series_df = [single_row_df(df_ts_features.iloc[i]) for i in range(len(df_ts_features))]

C:\dev\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
C:\dev\Anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\dev\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
C:\dev\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\dev\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\dev\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [160]:
df_series_feats = pd.concat(list_series_df)

In [161]:
df_series_feats.head()

,Validiy data,Persist. value_wind_2_amax,Persist. value_wind_2_amin,Persist. value_wind_2_std,Persist. value_wind_2_mean,Persist. value_wind_2_median,Persist. value_wind_2_trend,Persist. value_wind_2_mean_no_outliars,Persist. value_wind_2_rms_daily_change,CO_tmax_2_amax,...,EC_tmax_2_mean_no_outliars,EC_tmax_2_rms_daily_change,Persist. value_tmax_1_amax,Persist. value_tmax_1_amin,Persist. value_tmax_1_std,Persist. value_tmax_1_mean,Persist. value_tmax_1_median,Persist. value_tmax_1_trend,Persist. value_tmax_1_mean_no_outliars,Persist. value_tmax_1_rms_daily_change
0,1.514671e+09,18.0,6.0,4.130524,11.285714,9.0,0.500000,11.0,6.940221,21.0,...,18.2,2.081666,21.0,18.0,0.880631,19.714286,20.0,-0.285714,19.8,1.080123
0,1.514585e+09,18.0,6.0,4.070802,11.000000,9.0,0.714286,10.6,6.645801,21.0,...,18.2,2.198484,24.0,18.0,1.726149,20.142857,20.0,0.357143,19.8,1.914854
0,1.514498e+09,18.0,8.0,3.522522,11.857143,11.0,0.107143,11.4,6.544718,23.0,...,18.6,3.511885,24.0,18.0,1.989770,20.571429,20.0,0.642857,20.4,1.957890
0,1.514412e+09,18.0,8.0,3.522522,11.857143,11.0,-0.607143,11.4,5.552777,26.0,...,19.4,3.415650,24.0,18.0,2.138090,21.000000,20.0,0.821429,21.0,1.957890
0,1.514326e+09,16.0,8.0,2.490799,10.714286,10.0,0.071429,10.2,3.785939,26.0,...,20.6,3.391165,24.0,18.0,2.099563,21.142857,21.0,0.678571,21.2,2.081666


In [162]:
np.diff(df_series_feats['Validiy data'])

array([-86400., -86400., -86400., ..., -86400., -86400., -86400.])

In [163]:
df_series_feats.to_pickle('..//time_series_features.pickle')